In [5]:
!pip install casadi pymoo --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.6 MB/s eta 0:00:00


In [6]:
import casadi as ca
import numpy as np
from scipy.linalg import svd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.interpolate import interp1d
import time
import pandas as pd
from scipy.optimize import least_squares
import pandas as pd
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.core.sampling import Sampling
import numpy as np

In [7]:
# === Global Parameters and Configuration ===
param_names = [f"\u03b8{i+1}" for i in range(14)]
nx = 5
ny = 3
Tf = 161
Nt = 450
t_eval = np.linspace(0, Tf, Nt)

# === Nominal Parameters ===
theta_nominal = np.array([
    0.18, 0.225, 0.225, 0.01, 7.5, 7.5, 0.1737 / 4, 40, 0.00044,
    19.69, 1.60, 1.60, 0.49, 0.49
])
theta_low = 0.9 * theta_nominal
theta_high = 1.1 * theta_nominal

fixed_params = []
param_indices = [i for i in range(len(theta_nominal)) if i not in fixed_params]

# === Load Experimental Data ===

data_file = 'Bioreactor 2020 - cinética fermentación.xlsx'
sheet = 'LAB-LO(02)'

df = (
    pd.read_excel(data_file, sheet_name=sheet)
      .replace('-', np.nan)
      .dropna(subset=["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)", "Time (min)"])
)

df[["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)", "Time (min)"]] = df[["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)", "Time (min)"]].astype(float)
df["Time (h)"] = df["Time (min)"] / 60.0

# === 1b. Assign data to variables with expected names ===
t_exp        = df["Time (min)"].to_numpy(dtype=float) / 60.0   # min → hours
glucose_exp  = df["Glucosa (g/L)"].to_numpy(dtype=float)
fructose_exp = df["Fructosa (g/L)"].to_numpy(dtype=float)
yan_exp      = df["YAN (mg/L)"].to_numpy(dtype=float) / 1000.0 # mg/L → g/L
exp_data = df.groupby("Time (h)")[["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)"]].mean().reset_index()

# === Interpolate Experimental Data ===
y_exp_interp = np.zeros((len(t_eval), ny))
for j, col in enumerate(["YAN (mg/L)", "Glucosa (g/L)", "Fructosa (g/L)"]):
    y_exp_interp[:, j] = np.interp(t_eval, exp_data["Time (h)"], exp_data[col])

# === Initial Conditions ===
x0_val = np.array([0.2, 0.1737, 115.55, 100.94, 0.0])
dx0_val = np.zeros((nx, len(theta_nominal)))

def x0_func(*theta):
    return x0_val

def dx0_func(x0, theta):
    return dx0_val

# === Symbolic Model Definitions ===
x = ca.MX.sym("x", nx)
theta = ca.MX.sym("theta", 14)
U1 = ca.MX.sym("U1")
U2 = ca.MX.sym("U2")

th = [theta[i] for i in range(14)]
th1, th2, th3, th4, th5, th6, th7, th8, th9, th10, th11, th12, th13, th14 = th

# === Differential Equations ===
xdot = ca.vertcat(
    th1 * ca.exp(59453*(U1 - 300)/(300*8.314*U1)) * (x[1]/(x[1] + th4 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)))) * x[0]
    - U2 * th9 * ca.exp(0.0415 * x[4] + 130000*(U1 - 305.65)/(305.65*8.314*U1)) * x[0],

    -th1 * ca.exp(59453*(U1 - 300)/(300*8.314*U1)) * (x[1]/(x[1] + th4 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)))) * (x[0]/th10),

    -((th1 * ca.exp(59453*(U1 - 300)/(300*8.314*U1)) * (x[1]/(x[1] + th4 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)))) / th11)
      + (th2 * ca.exp(11000*(U1 - 296.15)/(296.15*8.314*U1)) * (x[2]/(x[2] + th5 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1))))
         * (th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)) / (x[4] + th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)))) / th13)
      + 0.01 * ca.exp(37681*(U1 - 293.3)/(293.3*8.314*U1)) * (x[2]/(x[2] + x[3]))) * x[0],

    -((th1 * ca.exp(59453*(U1 - 300)/(300*8.314*U1)) * (x[1]/(x[1] + th4 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)))) / th12)
      + (th3 * ca.exp(11000*(U1 - 296.15)/(296.15*8.314*U1)) * (x[3]/(x[3] + th6 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1))))
         * (th7 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)) / (x[2] + th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1))))
         * (th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)) / (x[4] + th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)))) / th14)
      + 0.01 * ca.exp(37681*(U1 - 293.3)/(293.3*8.314*U1)) * (x[3]/(x[2] + x[3]))) * x[0],

    (th2 * ca.exp(11000*(U1 - 296.15)/(296.15*8.314*U1)) * (x[2]/(x[2] + th5 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1))))
     * (th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)) / (x[4] + th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1))))
     + th3 * ca.exp(11000*(U1 - 296.15)/(296.15*8.314*U1)) * (x[3]/(x[3] + th6 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1))))
     * (th7 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)) / (x[2] + th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1))))
     * (th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1)) / (x[4] + th8 * ca.exp(46055*(U1 - 293.15)/(293.15*8.314*U1))))) * x[0]
)

# === Jacobians ===
dfdx = ca.jacobian(xdot, x)
dfdth = ca.jacobian(xdot, theta)

# === CasADi Functions ===
f_fun = ca.Function("f_fun", [x, U1, U2, theta], [xdot])
dfdx_fun = ca.Function("dfdx_fun", [x, U1, U2, theta], [dfdx])
dfdth_fun = ca.Function("dfdth_fun", [x, U1, U2, theta], [dfdth])
model_funcs = (f_fun, dfdx_fun, dfdth_fun)

# === Output Function ===
x_obs = ca.MX.sym("x_obs", nx)
theta_obs = ca.MX.sym("theta_obs", 14)
y_output = ca.vertcat(x_obs[1], x_obs[2], x_obs[3])
h_fun = ca.Function("h_fun", [x_obs, theta_obs], [y_output])

# === Meta Function for solve_ivp ===
def meta(t, x_ext, TU, theta_val, model_funcs, dims):
    nx, nth = dims
    f_fun, dfdx_fun, dfdth_fun = model_funcs
    X = x_ext[:nx]
    dx_dth = x_ext[nx:].reshape((nx, nth))

    U1_val = float(np.interp(t, TU[:, 0], TU[:, 1]))
    Td = -0.0001 * X[4]**3 + 0.0049 * X[4]**2 - 0.1279 * X[4] + 315.89
    U2_val = 1.0 if U1_val >= Td else 0.0

    dx = np.array(f_fun(X, U1_val, U2_val, theta_val)).flatten()
    A = np.array(dfdx_fun(X, U1_val, U2_val, theta_val))
    B = np.array(dfdth_fun(X, U1_val, U2_val, theta_val))
    d_dx_dth = A @ dx_dth + B

    return np.concatenate([dx, d_dx_dth.flatten()])

# === Residual Function ===
def residuals(theta_libres, *args):
    theta_full = theta_nominal.copy()
    theta_full[param_indices] = theta_libres

    try:
        x0 = np.array(x0_func(*theta_full)).flatten()
        dx0 = np.array(dx0_func(x0.reshape(-1, 1), theta_full)).reshape(x0.shape[0], -1)
        x_init = np.concatenate([x0, dx0.flatten()])

        sol = solve_ivp(
            lambda t, x: meta(t, x, TU, theta_full, model_funcs, (nx, len(theta_full))),
            (0, Tf), x_init,
            t_eval=t_eval,
            rtol=1e-6, atol=1e-8, method='BDF')

        if not sol.success:
            return np.ones(Nt * ny) * 1e6

        Xsol = sol.y[:nx, :].T
        y_model = np.zeros((Nt, ny))
        for i in range(Nt):
            y_model[i, :] = np.array(h_fun(Xsol[i], theta_full)).flatten()

        return (y_model - y_exp_interp).flatten()

    except Exception as e:
        print(f"[residuals] Error: {e}")
        return np.ones(Nt * ny) * 1e6

# === Load Operational Data ===
# === PART 6: Run Optimization ===
data = loadmat('Operational_Data.mat')
TU = data['Time_Temp_Pairs']
TU[:, 1] += 273.15  # Convert °C to K

# === T-Value Analysis Function ===
def analyze_t_values(residuals_func, p0, bounds, texp, ydata, fixed_idx=None, n_iter=10):
    n_params = len(p0)
    t_values_all = []

    for i in range(n_iter):
        p_init = np.random.uniform(bounds[0], bounds[1])
        if fixed_idx is not None:
            p_init[fixed_idx] = p0[fixed_idx]

        result = least_squares(residuals_func, p_init, args=(texp, ydata), bounds=bounds, jac='3-point')

        params_hat = result.x
        J = result.jac
        resid = result.fun
        sigma2 = np.sum(resid**2) / (len(ydata) - np.linalg.matrix_rank(J))

        try:
            Cov_theta = sigma2 * np.linalg.pinv(J.T @ J)
            std_theta = np.sqrt(np.diag(Cov_theta))
            t_values = params_hat / std_theta
            t_values_all.append(t_values)
        except np.linalg.LinAlgError:
            print(f"[{i+1}] Singular matrix. Skipping iteration.")
            continue

    t_values_all = np.array(t_values_all)
    non_identifiable = (np.abs(t_values_all) < 1.96)
    count_non_identifiable = np.sum(non_identifiable, axis=0)

    print("\n📊 Count of times each parameter was non-identifiable (|t| < 1.96):")
    for i, count in enumerate(count_non_identifiable):
        print(f"\u03b8{param_indices[i]+1}: {count} of {t_values_all.shape[0]} iterations → {'❌' if count > 0 else '✅'}")

    t_avg = np.nanmean(t_values_all, axis=0)
    t_std = np.nanstd(t_values_all, axis=0)
    return t_avg, t_std, t_values_all

<ipython-input-7-a26deba015fc>:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace('-', np.nan)


In [ ]:
# === Optimization Problem Class for Practical Identifiability (T-values) ===
class TValueIdentifiabilityProblem(Problem):
    def __init__(self, theta_nom, residuals_func, bounds, t_eval, y_exp_interp, sigma_t=1.96, n_iter=5):
        super().__init__(n_var=len(theta_nom), n_obj=1, xl=0, xu=1, type_var=int)
        self.theta_nom = theta_nom
        self.residuals_func = residuals_func
        self.bounds = bounds
        self.texp = t_eval
        self.ydata = y_exp_interp.flatten()
        self.t_threshold = sigma_t
        self.n_iter = n_iter

        self.cache = set()
        self.successful_combinations = set()
        self.log_combinations = []  # 🧠 stores (mask, t_avg, score)

    def is_pruned_combination(self, fixed_idx):
        return any(len(fixed_idx) >= len(success) for success in self.successful_combinations)

    def _evaluate(self, X, out, *args, **kwargs):
        F = np.full((X.shape[0],), 1e6)

        for i in range(X.shape[0]):
            mask = tuple(int(val) for val in X[i])
            mask_bool = np.array(mask).astype(bool)
            idx_free = [j for j in range(len(mask)) if mask_bool[j]]
            idx_fixed = [j for j in range(len(mask)) if not mask_bool[j]]

            if mask in self.cache:
                print(f"🔁 Combination already evaluated. Skipping.")
                F[i] = 1e6
                continue
            self.cache.add(mask)

            if self.is_pruned_combination(idx_fixed):
                print(f"⏭️ Pruned: too many fixed parameters.")
                F[i] = 1e6
                continue

            tvals, score = evaluate_combination_tvalues(
                mask=mask,
                theta_nominal=self.theta_nom,
                residuals_func=self.residuals_func,
                bounds=self.bounds,
                t_eval=self.texp,
                y_exp_interp=self.ydata,
                t_threshold=self.t_threshold,
                n_iter=self.n_iter
            )

            # 💾 Store the result
            self.log_combinations.append({
                "mask": mask,
                "t_avg": tvals,
                "score": score
            })

            if tvals is not None:
                n_bad = np.sum(np.abs(tvals) < self.t_threshold)
                if n_bad == 0:
                    print(f"✅ Valid combination. Adding pruning rule for future generations.")
                    print(f"🧠 Future combinations with ≥ {len(idx_fixed)} fixed parameters will be skipped.")
                    print(f"   ↳ Current fixed: {[f'θ{j+1}' for j in idx_fixed]}")
                    self.successful_combinations.add(tuple(sorted(idx_fixed)))

            F[i] = score if score is not None else 1e6

        out["F"] = F

# === Evaluator for a fixed parameter combination ===
def evaluate_combination_tvalues(mask, theta_nominal, residuals_func, bounds, t_eval, y_exp_interp, t_threshold=1.96, n_iter=5):
    mask = np.array(mask).astype(bool)
    idx_free = [i for i in range(len(theta_nominal)) if mask[i]]
    idx_fixed = [i for i in range(len(theta_nominal)) if not mask[i]]

    if len(idx_free) == 0:
        print("⛔️ All parameters are fixed. Invalid combination.")
        return None, 1e6

    print(f"\n🧪 Evaluating combination:")
    print(f"   ↳ Free: {[f'θ{i+1}' for i in idx_free]}")
    print(f"   ↳ Fixed: {[f'θ{i+1}' for i in idx_fixed]}")

    def masked_residuals(theta_free, *args):
        theta_full = theta_nominal.copy()
        theta_full[idx_free] = theta_free
        return residuals(theta_full, *args)

    try:
        t_avg, t_std, t_all = analyze_t_values(
            residuals_func=masked_residuals,
            p0=theta_nominal[idx_free],
            bounds=(bounds[0][idx_free], bounds[1][idx_free]),
            texp=t_eval,
            ydata=y_exp_interp,
            fixed_idx=None,
            n_iter=n_iter
        )

        n_bad = np.sum(np.abs(t_avg) < t_threshold)
        score = len(idx_fixed) + 10 * n_bad  # penalize fixed + unidentifiable

        print(f"📌 Average t-values: {np.round(t_avg, 2)}")
        print(f"❗️ {n_bad} non-identifiable (|t| < {t_threshold})")
        return t_avg, score

    except Exception as e:
        print(f"❌ Error in combination: {e}")
        return None, 1e6

# === Custom Sampling: starts with all parameters free ===
class CustomSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        pop = np.random.randint(0, 2, size=(n_samples - 1, problem.n_var))
        start = np.ones(problem.n_var, dtype=int).reshape(1, -1)  # all free
        return np.vstack([start, pop])

# === GENERAL CONFIGURATION ===
n_iter = 1  # how many t-value replicates per combination
pop_size = 20
n_generations = 5
theta_nom = theta_nominal  # previously defined
bounds = (theta_low, theta_high)  # previously defined

# === RUN THE OPTIMIZATION ===
problem = TValueIdentifiabilityProblem(
    theta_nom=theta_nom,
    residuals_func=residuals,
    bounds=bounds,
    t_eval=t_eval,
    y_exp_interp=y_exp_interp,
    sigma_t=1.96,
    n_iter=n_iter
)

algorithm = GA(
    pop_size=pop_size,
    sampling=CustomSampling(),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", n_generations)

res = minimize(
    problem,
    algorithm,
    termination,
    seed=42,
    verbose=True
)

# === FINAL RESULT ===
best = res.X
idx_free = [i for i in range(len(theta_nom)) if best[i] == 1]
idx_fixed = [i for i in range(len(theta_nom)) if best[i] == 0]

print("\n✅ Best combination found (free parameters):", idx_free)
print("   ↳ Fixed:", idx_fixed)
import matplotlib.pyplot as plt

# 🧠 Recover the t-values of the best combination
best_mask = tuple(int(val) for val in best)
log_entry = next((entry for entry in problem.log_combinations if entry["mask"] == best_mask), None)

if log_entry and log_entry["t_avg"] is not None:
    t_avg = log_entry["t_avg"]
    param_names = [f"θ{i+1}" for i in range(len(theta_nom))]

    plt.figure(figsize=(10, 5))
    bars = plt.bar(param_names, t_avg)
    plt.axhline(y=1.96, color='r', linestyle='--', label='Threshold |t| = 1.96')
    plt.axhline(y=-1.96, color='r', linestyle='--')
    plt.xticks(rotation=45)
    plt.title("Average t-values for Best Combination")
    plt.ylabel("t-value")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Could not retrieve t-values for the best combination.")



🧪 Evaluating combination:
   ↳ Free: ['θ1', 'θ2', 'θ3', 'θ4', 'θ5', 'θ6', 'θ7', 'θ8', 'θ9', 'θ10', 'θ11', 'θ12', 'θ13', 'θ14']
   ↳ Fixed: []
